In [1]:
from dataclasses import dataclass
import shelve
from datetime import datetime
import random
from scripts import utils
from pprint import pprint
import hashlib
import os
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import DistilBertModel, BertModel
from transformers import LEDForConditionalGeneration, LEDTokenizer
import dist
import importlib


importlib.reload(dist)
plt.style.use('dark_background')

In [2]:
DF_PATH = './benchmarking-dataset/runtime-data.parquet'
DEVICE = 'cuda'

## Data Visualization

In [3]:
df = pl.read_parquet(DF_PATH) 
df = df.sort(pl.col('runtime'))
df

c_filename,flags,runtime,created_at,trials,sys_specs_hash,sys_specs_os,sys_specs_cpu,sys_specs_compiler,sys_specs_os_version,sys_specs_cpu_arch,sys_specs_cpu_physical_cores,sys_specs_cpu_logical_cores,sys_specs_mem_size_gb,sys_specs_mem_speed_mhz,sys_specs_l1_i_size,sys_specs_l1_d_size,sys_specs_l2_size,sys_specs_l3_size,id
str,str,f64,datetime[μs],i64,str,str,str,str,str,str,i64,i64,f64,null,str,str,str,str,str
"""eigenvalue-33c06ce6.c""","""-O3 -march=native -mtune=nativ…",0.304516,2024-11-03 22:30:10.171520,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""948bddda92e7f5e374c0cfa1049165…"
"""eigenvalue-33c06ce6.c""","""-O3 -funroll-loops -fprefetch-…",0.304858,2024-11-03 22:29:59.796546,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""fa886d31e2da16d92dea8e4bed3c39…"
"""eigenvalue-33c06ce6.c""","""-O3 -funroll-loops -march=nati…",0.41399,2024-11-03 22:29:59.449327,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""9bef08bfb4fff715a387f79d80c043…"
"""eigenvalue-33c06ce6.c""","""-O3 -march=native -funroll-loo…",0.414015,2024-11-03 22:29:50.755481,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""52a4971819d05b87a67712f029741b…"
"""eigenvalue-33c06ce6.c""","""-O3 -ffast-math -funroll-loops…",0.41501,2024-11-03 22:29:57.171162,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""2fd2e51049c878750c7cb46ef0de9c…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""cond-branching-87209a89.c""","""-O3 -flto -fwhole-program -mar…",4.4576,2024-11-03 21:43:32.089278,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""c4b3cb18925e849d4cc904262ff496…"
"""cond-branching-87209a89.c""","""-O3 -mtune=native -funroll-loo…",4.467,2024-11-03 21:42:57.122584,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""947b96015fe6d7ad79eb5ad66391e7…"
"""cond-branching-87209a89.c""","""-O3 -funroll-loops -flto""",4.469425,2024-11-03 21:41:33.502398,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""28b31a0705d2555865059d86cb0425…"


In [4]:
df_grouped = df.sort(pl.col('runtime')).group_by(pl.col('c_filename')).first()
df_grouped = df_grouped.sort(pl.col('c_filename'))
df_grouped

c_filename,flags,runtime,created_at,trials,sys_specs_hash,sys_specs_os,sys_specs_cpu,sys_specs_compiler,sys_specs_os_version,sys_specs_cpu_arch,sys_specs_cpu_physical_cores,sys_specs_cpu_logical_cores,sys_specs_mem_size_gb,sys_specs_mem_speed_mhz,sys_specs_l1_i_size,sys_specs_l1_d_size,sys_specs_l2_size,sys_specs_l3_size,id
str,str,f64,datetime[μs],i64,str,str,str,str,str,str,i64,i64,f64,null,str,str,str,str,str
"""b-tree-740a1306.c""","""-O3 -march=native""",3.7287985,2024-11-03 21:16:19.616530,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""a68bc1164e35fcf2eda87b76d589df…"
"""cond-branching-87209a89.c""","""-O3 -funroll-loops -march=nati…",4.2437,2024-11-03 21:43:23.313757,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""3bc1926cb52466e6bf61eda36a4d3d…"
"""dijkstra-8d333a4f.c""","""-O3 -flto""",1.2221225,2024-11-03 21:32:05.412505,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""4528e47a3b92fc6b5a4fa5000a20c5…"
"""eigenvalue-33c06ce6.c""","""-O3 -march=native -mtune=nativ…",0.304516,2024-11-03 22:30:10.171520,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""948bddda92e7f5e374c0cfa1049165…"
"""guass-elimination-3a6b05fc.c""","""-O3 -flto -fwhole-program -mar…",1.080903,2024-11-03 21:55:14.925916,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""25728dd145beba4c686d77b7c2e5c6…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""mergesort-a32154f4.c""","""-O3 -march=native -fomit-frame…",1.850719,2024-11-03 22:11:16.193518,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""ce6ea2366c92f3f50c44ea8c97afba…"
"""n-bodies-753227e0.c""","""-O3 -fwhole-program -funroll-l…",1.296985,2024-11-03 20:17:03.037302,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""faed48cafea58e5424bb71b619e985…"
"""quick-sort-d9140517.c""","""-O3 -funroll-loops -fno-math-e…",1.520188,2024-11-03 22:23:58.084474,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""e70a3ef19865dcde5d89e76e3d8f83…"


## Data Preperation

In [5]:
X_raw = []
y = []

iter_obj = zip(
    df_grouped['sys_specs_cpu'].to_list(),
    df_grouped['sys_specs_os_version'].to_list(),
    df_grouped['sys_specs_cpu_arch'].to_list(),
    df_grouped['sys_specs_compiler'].to_list(),
    df_grouped['c_filename'].to_list(),
	df_grouped['flags'].to_list(),
)

for (cpu, os, arch, compiler, c_file, flags) in iter_obj:
	with open(f'C-dataset/{c_file}', 'r') as f:
		c_source = f.read()
	payload = dist.torch_utils.InferenceUtils.format_prompt(c_source, cpu, arch, os, compiler)
	X_raw.append(payload)
	y.append(dist.torch_utils.InferenceUtils.onhot_encode(flags))

y = torch.from_numpy(np.stack(y))


In [6]:
# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
# hf_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

# model_name = "allenai/led-base-16384"  # LED model with a 16,384 token context window
# tokenizer = LEDTokenizer.from_pretrained(model_name)
# hf_model = LEDForConditionalGeneration.from_pretrained(model_name)

In [7]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
hf_model = DistilBertModel.from_pretrained("distilbert-base-uncased")

In [8]:
X = tokenizer(X_raw, padding='max_length', return_tensors='pt', max_length=(2**12))
X = {k:v for k, v in X.items()}
{k:v.size() for k, v in X.items()}

{'input_ids': torch.Size([17, 4096]), 'attention_mask': torch.Size([17, 4096])}

## Model Training

In [9]:
X = {k:v.to(DEVICE) for k, v in X.items()}
y = y.to(DEVICE).to(torch.float32)

In [10]:
model = dist.torch_utils.FlagModel(hf_model, len(dist.torch_utils.InferenceUtils.FLAGS)).to(DEVICE)

In [13]:
# 1) Set up loss function and optimizer
loss = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW([
    {'params': model.language_model.parameters(),  'lr': 1e-6},
	{'params': model.fc_in.parameters(), 'lr': 1e-3},
    {'params': model.fc1.parameters(),   'lr': 1e-3},
	{'params': model.fc2.parameters(),   'lr': 1e-3},
    {'params': model.out.parameters(),   'lr': 1e-3},
])

model.train() # Set model to train mode (turn on gradient tracking)

epochs = 45
for epoch in range (epochs):
	# 2) Forward Propagation
	y_pred = model(X)

	# 3) loss calculation
	curr_loss = loss(y_pred, y)

	# 4) Clear out optimizer gradients
	optimizer.zero_grad()

	# 5) Back Propagation
	curr_loss.backward()

	# 6) Optimizer step
	optimizer.step()

	print(f"Epoch: {epoch:<3} : {curr_loss}")

Epoch: 0   : 3.467581272125244
Epoch: 1   : 1.4357578754425049
Epoch: 2   : 0.8747193813323975
Epoch: 3   : 0.7802227139472961
Epoch: 4   : 0.7162343263626099
Epoch: 5   : 0.672146737575531
Epoch: 6   : 0.6354959011077881
Epoch: 7   : 0.6114402413368225
Epoch: 8   : 0.5894307494163513
Epoch: 9   : 0.5748398900032043
Epoch: 10  : 0.5647257566452026
Epoch: 11  : 0.5561930537223816
Epoch: 12  : 0.5479611754417419
Epoch: 13  : 0.5387503504753113
Epoch: 14  : 0.530734121799469
Epoch: 15  : 0.5227965116500854
Epoch: 16  : 0.5173941254615784
Epoch: 17  : 0.5117778778076172
Epoch: 18  : 0.5065511465072632
Epoch: 19  : 0.5014582872390747
Epoch: 20  : 0.4943024218082428
Epoch: 21  : 0.48640143871307373
Epoch: 22  : 0.47873178124427795
Epoch: 23  : 0.46921348571777344
Epoch: 24  : 0.46031415462493896
Epoch: 25  : 0.45124778151512146
Epoch: 26  : 0.44281989336013794
Epoch: 27  : 0.4356633424758911
Epoch: 28  : 0.42823702096939087
Epoch: 29  : 0.42099443078041077
Epoch: 30  : 0.4132651686668396
Epo

In [ ]:
torch.save(model, './dist/models/trufling-flags-70M.pt')

## Testing

In [18]:
model = torch.load("./dist/models/trufling-70M.pt")

file = 'C-dataset/linkedlist-78b31e23.c'
file = 'C-dataset/eigenvalue-33c06ce6.c'

with open(file, 'r') as f:
    c_source = f.read()

tokens = tokenizer([c_source], return_tensors='pt', padding='max_length', max_length=4096)
tokens = {k:v.to(DEVICE) for k, v in tokens.items()}


with torch.no_grad():
    res = model(tokens)
    res = res.flatten()
    res = torch.sigmoid(res)
print(res.size(), '\n')

print(res, '\n')

flags = dist.torch_utils.InferenceUtils.gen_flags(res)
print(flags)
print('\n\n\n')

torch.Size([16]) 

tensor([0.9985, 0.9276, 0.6222, 0.8843, 0.8120, 0.9758, 0.8824, 0.9610, 0.3903,
        0.9869, 0.1565, 0.9967, 0.7015, 0.9938, 0.9943, 0.8292],
       device='cuda:0') 

['-O3', '-falign-functions=32', '-falign-loops=32', '-ffast-math', '-fipa-pta', '-flto', '-fmerge-all-constants', '-fno-math-errno', '-fomit-frame-pointer', '-fprefetch-loop-arrays', '-ftree-vectorize', '-funroll-loops', '-funsafe-math-optimizations', '-fwhole-program', '-march=native', '-mtune=native']






/tmp/ipykernel_18848/2676800994.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("./dist/models/trufling-70M.pt")


In [24]:
def count_parameters(model):
    total_params = 0
    for param in model.parameters():
        if param.requires_grad:
            total_params += param.numel()
    return total_params

count_parameters(model)

69010000

In [46]:
s = df_grouped[['c_filename', 'flags']].to_struct().to_list()
for i in s:
    print(f"Filename: {i['c_filename']}\nFlag: {i['flags']}\n")

Filename: b-tree-740a1306.c
Flag: -O3 -march=native

Filename: cond-branching-87209a89.c
Flag: -O3 -funroll-loops -march=native -mtune=native -fno-math-errno -ffast-math

Filename: dijkstra-8d333a4f.c
Flag: -O3 -flto

Filename: eigenvalue-33c06ce6.c
Flag: -O3 -march=native -mtune=native -funroll-loops -fprefetch-loop-arrays -flto -ffast-math -fno-math-errno -funsafe-math-optimizations -fomit-frame-pointer -fwhole-program -fmerge-all-constants -fipa-pta -falign-functions=32 -falign-loops=32

Filename: guass-elimination-3a6b05fc.c
Flag: -O3 -flto -fwhole-program -march=native -funroll-loops

Filename: hashmap-d3e0d305.c
Flag: -O3 -funroll-loops -march=native -mtune=native -fno-math-errno -ffast-math

Filename: heapsort-43e011c8.c
Flag: -O3 -fprefetch-loop-arrays

Filename: json-parsing-46a4d7b3.c
Flag: -O3 -funroll-loops -fprefetch-loop-arrays -march=native -ffast-math

Filename: knapsack-758bb0aa.c
Flag: -O3 -march=native -mtune=native -funroll-loops -fprefetch-loop-arrays -flto -ffast-

## Scratch

In [2]:
utils.SystemSpec.generate()

SystemSpec(os='Linux', cpu='AMD Ryzen 9 3950X 16-Core Processor', compiler='gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0', os_version='Linux-6.9.3-76060903-generic-x86_64-with-glibc2.35', cpu_arch='x86_64', cpu_physical_cores=16, cpu_logical_cores=32, mem_size_gb=32.780768, mem_speed_mhz=None, l1_i_size='512 KiB (16 instances)', l1_d_size='512 KiB (16 instances)', l2_size='8 MiB (16 instances)', l3_size='64 MiB (4 instances)')

In [ ]:
with shelve.open('./benchmarking-dataset/benchmark.shelve') as db:
    data = dict(db)
len(data)

In [ ]:
idx = 9
ptr: utils.ExecutionTrial = list(data.values())[idx]
print(ptr.c_filename)
print(ptr.flags)
print(ptr.time_elapsed())

In [20]:
import requests
import json

# Define the text you want to embed
text = "Your text here."

# Set up the request payload
payload = {
    "model": "nomic-embed-text",
    "prompt": c_source,    
}

# Send the request to the Ollama API
response = requests.post("http://localhost:11434/api/embeddings", json=payload)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    # Access the embedding
    embedding = data.get("embedding")
    print("Embedding:", embedding)
else:
    print(f"Error: {response.status_code} - {response.text}")


Embedding: [1.6490572690963745, 0.9081770777702332, -1.1945759057998657, -0.5772809386253357, 0.16351068019866943, -0.3475622832775116, 1.140226125717163, -0.17751000821590424, 0.5545435547828674, -0.7910482883453369, -1.219162106513977, 0.2671194076538086, 1.1699737310409546, 0.31394606828689575, 0.19645237922668457, -0.5049535632133484, -0.38436177372932434, -0.987790584564209, 0.6696007251739502, 0.593305766582489, 0.3193209767341614, 0.023933107033371925, -0.39659640192985535, 0.029780719429254532, -0.21617330610752106, -0.013321535661816597, -0.6268774271011353, -0.11695615947246552, -1.0875308513641357, -0.07231128215789795, 0.21316155791282654, 0.07577144354581833, 0.36072975397109985, -0.035064712166786194, -0.7211555242538452, 0.12904375791549683, 0.4178484380245209, -0.1826760172843933, 1.2785347700119019, -0.2995186448097229, -0.08250722289085388, 0.25382620096206665, 0.15698009729385376, 0.18455518782138824, 0.539380669593811, 0.11845489591360092, 1.109390139579773, 0.35356

In [17]:
print(
    type(embedding),
    len(embedding),
)

<class 'list'> 768
